In [1]:
import pandas as pd
import numpy as np
import random
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import seaborn as sns

# Set random seed for reproducible results
np.random.seed(42)
random.seed(42)

print("🚗 Creating Uber Dataset for Indian Cities!")
print("="*50)

# Step 1: Create Indian market ride data
def create_indian_ride_data(n_rides=1000):
    """
    Create a realistic dataset for Indian Uber market
    Key differences: Higher price sensitivity, different peak hours, weather impact
    """
    
    # Major Indian cities with Uber presence
    cities = ['Mumbai', 'Delhi', 'Bangalore', 'Hyderabad', 'Chennai', 'Kolkata', 'Pune', 'Ahmedabad']
    
    # City characteristics (base fare varies by city)
    city_base_fares = {
        'Mumbai': {'min': 15, 'max': 45},  # Higher due to expensive city
        'Delhi': {'min': 12, 'max': 35},
        'Bangalore': {'min': 10, 'max': 30},
        'Hyderabad': {'min': 8, 'max': 25},
        'Chennai': {'min': 8, 'max': 25},
        'Kolkata': {'min': 7, 'max': 22},
        'Pune': {'min': 9, 'max': 28},
        'Ahmedabad': {'min': 7, 'max': 20}
    }
    
    # Weather conditions common in India
    weather_conditions = ['Clear', 'Light_Rain', 'Heavy_Rain', 'Hot', 'Humid', 'Dusty']
    
    # Indian festivals/events (simplified)
    special_days = ['Diwali', 'Holi', 'Eid', 'Regular', 'Regular', 'Regular', 'Regular']  # Most days are regular
    
    rides = []
    
    for i in range(n_rides):
        city = random.choice(cities)
        
        # Basic ride information
        ride = {
            'ride_id': f'IND_{i+1:04d}',
            'city': city,
            'hour': random.randint(0, 23),
            'day_of_week': random.randint(1, 7),  # 1=Monday, 7=Sunday
            'weather': random.choice(weather_conditions),
            'special_event': random.choice(special_days),
        }
        
        # Base fare based on city and distance
        city_fare_range = city_base_fares[city]
        ride['distance_km'] = round(random.uniform(2, 25), 1)  # Using KM for India
        ride['base_fare'] = round(random.uniform(city_fare_range['min'], city_fare_range['max']), 2)
        
        # Indian peak hours are different
        morning_peak = ride['hour'] in [8, 9, 10]  # Office going
        evening_peak = ride['hour'] in [18, 19, 20, 21]  # Office return + dinner
        late_night = ride['hour'] in [22, 23, 0, 1]  # Party/late dining
        weekend_evening = ride['day_of_week'] in [6, 7] and ride['hour'] in [19, 20, 21, 22]
        
        # Weather impact (very important in India)
        weather_surge = 1.0
        if ride['weather'] == 'Heavy_Rain':
            weather_surge = 1.5  # High demand during heavy rain
        elif ride['weather'] == 'Light_Rain':
            weather_surge = 1.2
        elif ride['weather'] == 'Hot' and ride['hour'] in [12, 13, 14, 15]:
            weather_surge = 1.3  # Afternoon heat
        
        # Festival surge
        festival_surge = 1.0
        if ride['special_event'] in ['Diwali', 'Holi', 'Eid']:
            festival_surge = 1.8  # High demand during festivals
        
        # Base surge calculation
        if late_night:
            base_surge = random.choice([1.5, 2.0, 2.5])
        elif evening_peak:
            base_surge = random.choice([1.2, 1.5, 2.0])
        elif morning_peak:
            base_surge = random.choice([1.0, 1.2, 1.5])
        elif weekend_evening:
            base_surge = random.choice([1.3, 1.8, 2.2])
        else:
            base_surge = random.choice([1.0, 1.0, 1.0, 1.1])  # Normal times
        
        # Combine all surge factors
        total_surge = base_surge * weather_surge * festival_surge
        
        # Cap surge at realistic Indian levels (Indian customers are price sensitive)
        ride['surge_multiplier'] = min(total_surge, 3.5)  # Max 3.5x surge
        ride['final_fare'] = round(ride['base_fare'] * ride['surge_multiplier'], 2)
        
        # Indian customer behavior - MUCH more price sensitive
        cancel_probability = 0.03  # Base 3% cancellation rate
        
        # Indians are very price sensitive
        if ride['surge_multiplier'] >= 3.0:
            cancel_probability = 0.60  # 60% cancel at 3x+ surge!
        elif ride['surge_multiplier'] >= 2.5:
            cancel_probability = 0.45  # 45% cancel at 2.5x surge
        elif ride['surge_multiplier'] >= 2.0:
            cancel_probability = 0.30  # 30% cancel at 2x surge
        elif ride['surge_multiplier'] >= 1.5:
            cancel_probability = 0.15  # 15% cancel at 1.5x surge
        elif ride['surge_multiplier'] >= 1.3:
            cancel_probability = 0.08  # 8% cancel at 1.3x surge
        
        # Exception: During heavy rain, people are less price sensitive
        if ride['weather'] == 'Heavy_Rain':
            cancel_probability *= 0.7  # 30% less likely to cancel in rain
        
        # Exception: During festivals, people are less price sensitive
        if ride['special_event'] in ['Diwali', 'Holi', 'Eid']:
            cancel_probability *= 0.6  # 40% less likely to cancel during festivals
        
        ride['ride_completed'] = random.random() > cancel_probability
        
        # Wait time calculation (longer waits in Indian traffic)
        base_wait = 8  # Longer base wait time in Indian cities
        surge_wait = (ride['surge_multiplier'] - 1) * 4
        traffic_factor = 2 if ride['hour'] in [8, 9, 18, 19, 20] else 0
        weather_wait = 5 if ride['weather'] == 'Heavy_Rain' else 0
        
        ride['wait_time_mins'] = max(2, round(base_wait + surge_wait + traffic_factor + weather_wait + random.uniform(-3, 5)))
        
        # Add customer segment (important in Indian market)
        ride['customer_segment'] = random.choices(
            ['Budget', 'Regular', 'Premium'], 
            weights=[50, 35, 15],  # Most Indians are budget conscious
            k=1
        )[0]
        
        # Adjust cancellation based on customer segment
        if ride['customer_segment'] == 'Budget' and ride['surge_multiplier'] > 1.5:
            if random.random() < 0.3:  # 30% more likely to cancel
                ride['ride_completed'] = False
        elif ride['customer_segment'] == 'Premium':
            if random.random() < 0.15:  # Premium customers less likely to cancel
                ride['ride_completed'] = True
        
        rides.append(ride)
    
    return pd.DataFrame(rides)

# Generate Indian market dataset!
df = create_indian_ride_data(1500)  # Larger dataset

print(f"✅ Created Indian market dataset with {len(df)} rides")
print(f"📊 Cities covered: {', '.join(df['city'].unique())}")
print(f"📊 Columns: {list(df.columns)}")

print("\n🔍 First 5 rides:")
print(df.head())

print("\n📈 Indian Market Statistics:")
print(f"Average base fare: ₹{df['base_fare'].mean():.2f}")
print(f"Average surge multiplier: {df['surge_multiplier'].mean():.2f}x")
print(f"Overall completion rate: {df['ride_completed'].mean()*100:.1f}%")
print(f"Weather impact rides: {len(df[df['weather'].isin(['Light_Rain', 'Heavy_Rain'])])} rides")

# City-wise analysis
city_analysis = df.groupby('city').agg({
    'base_fare': 'mean',
    'surge_multiplier': 'mean',
    'ride_completed': 'mean',
    'ride_id': 'count'
}).round(2)
city_analysis.columns = ['Avg_Base_Fare', 'Avg_Surge', 'Completion_Rate', 'Total_Rides']

print("\n🏙️ CITY-WISE ANALYSIS:")
print("="*50)
print(city_analysis)

# Weather impact analysis
weather_analysis = df.groupby('weather').agg({
    'surge_multiplier': 'mean',
    'ride_completed': 'mean',
    'final_fare': 'mean'
}).round(2)

print("\n🌧️ WEATHER IMPACT ANALYSIS:")
print("="*35)
print(weather_analysis)

# Customer segment analysis
segment_analysis = df.groupby('customer_segment').agg({
    'surge_multiplier': 'mean',
    'ride_completed': 'mean',
    'final_fare': 'mean',
    'ride_id': 'count'
}).round(2)
segment_analysis.columns = ['Avg_Surge_Accepted', 'Completion_Rate', 'Avg_Fare_Paid', 'Total_Rides']

print("\n👥 CUSTOMER SEGMENT ANALYSIS:")
print("="*40)
print(segment_analysis)

# Save the Indian data
df.to_csv('uber_rides_india.csv', index=False)
print(f"\n💾 Saved Indian market data to 'uber_rides_india.csv'")

print("\n🚀 INDIAN MARKET INSIGHTS:")
print("="*35)
print("• Indians are MUCH more price sensitive than global markets")
print("• Weather (especially rain) has huge impact on demand")
print("• Festival days see different pricing acceptance")
print("• Budget customers dominate the market (50%)")
print("• Traffic and wait times are significant factors")

print("\n🔥 NEXT STEPS FOR INDIAN MARKET:")
print("1. Analyze optimal surge by city (Mumbai vs Kolkata)")
print("2. Weather-based pricing strategy")
print("3. Festival demand patterns")
print("4. Customer segment-specific strategies")
print("5. Ready for Indian market visualization? Let me know!")

🚗 Creating Uber Dataset for Indian Cities!
✅ Created Indian market dataset with 1500 rides
📊 Cities covered: Delhi, Mumbai, Chennai, Kolkata, Ahmedabad, Hyderabad, Pune, Bangalore
📊 Columns: ['ride_id', 'city', 'hour', 'day_of_week', 'weather', 'special_event', 'distance_km', 'base_fare', 'surge_multiplier', 'final_fare', 'ride_completed', 'wait_time_mins', 'customer_segment']

🔍 First 5 rides:
    ride_id     city  hour  day_of_week     weather special_event  \
0  IND_0001    Delhi     0            6  Heavy_Rain          Holi   
1  IND_0002   Mumbai     0            1  Light_Rain          Holi   
2  IND_0003  Chennai     0            7  Light_Rain       Regular   
3  IND_0004  Kolkata    19            3       Clear       Regular   
4  IND_0005  Kolkata    18            2       Dusty        Diwali   

   distance_km  base_fare  surge_multiplier  final_fare  ride_completed  \
0          7.1      28.94              3.50      101.29            True   
1         13.6      15.80            